In [1]:
#Libs
import matplotlib.pyplot as plt
import numpy as np
import pylab
import pandas as pd
import h5py
import geopandas as gpd
from shapely.geometry import *
from geopandas.geoseries import *
import sys
from scipy.interpolate import make_interp_spline, BSpline
from scipy import misc
from scipy.ndimage import gaussian_filter

test = 'D:/Gabon_Lidar/IDL/Default/LVISDATA/New_England_LVISDATA/LVIS_US_ME_2009_VECT_20100328/LVIS_US_ME_2009_VECT_20100328/LVIS_US_ME_2009_VECT_20100328.lce'
test1 = 'D:/Gabon_Lidar/IDL/Default/LVISDATA/New_England_LVISDATA/LVIS_US_ME_2009_VECT_20100328/LVIS_US_ME_2009_VECT_20100328/LVIS_US_ME_2009_VECT_20100328.lge'
test2 = 'D:/Gabon_Lidar/IDL/Default/LVISDATA/New_England_LVISDATA/LVIS_US_ME_2009_VECT_20100328/LVIS_US_ME_2009_VECT_20100328/LVIS_US_ME_2009_VECT_20100328.lgw'

DESCRIPTION = '''
Read LVIS binary files generated from 1998 to 2009.

Formats:
    .lce
    .lge
    .lgw

    Data versions 1.00 to 1.04

LVIS Operations Team
Sarah Story
2020/7/17
'''

import numpy as np


def get_datatype(filetype, version):
    if filetype == 'lce':
        if version == '1.00':
            dt = np.dtype([('tlon', '>f8'), ('tlat', '>f8'), ('zt', '>f4')])
        elif version == '1.01':
            dt = np.dtype([('lfid', '>u4'), ('shotnumber', '>u4'), ('tlon', '>f8'), ('tlat', '>f8'), ('zt', '>f4')])
        elif version == '1.02':
            dt = np.dtype([('lfid', '>u4'), ('shotnumber', '>u4'), ('time', '>f8'), ('tlon', '>f8'), ('tlat', '>f8'),
                           ('zt', '>f4')])
        elif version == '1.03':
            dt = np.dtype(
                [('lfid', '>u4'), ('shotnumber', '>u4'), ('azimuth', '>f4'), ('incidentangle', '>f4'), ('range', '>f4'),
                 ('time', '>f8'), ('tlon', '>f8'), ('tlat', '>f8'), ('zt', '>f4')])
        elif version == '1.04':
            dt = np.dtype(
                [('lfid', '<u4'), ('shotnumber', '<u4'), ('azimuth', '<f4'), ('incidentangle', '<f4'), ('range', '<f4'),
                 ('time', '<f8'), ('tlon', '<f8'), ('tlat', '<f8'), ('zt', '<f4')])


    elif filetype == 'lge':
        if version == '1.00':
            dt = np.dtype(
                [('glon', '>f8'), ('glat', '>f8'), ('zg', '>f4'), ('rh25', '>f4'), ('rh50', '>f4'), ('rh75', '>f4'),
                 ('rh100', '>f4')])
        elif version == '1.01':
            dt = np.dtype([('lfid', '>u4'), ('shotnumber', '>u4'), ('glon', '>f8'), ('glat', '>f8'), ('zg', '>f4'),
                           ('rh25', '>f4'), ('rh50', '>f4'), ('rh75', '>f4'), ('rh100', '>f4')])
        elif version == '1.02':
            dt = np.dtype([('lfid', '>u4'), ('shotnumber', '>u4'), ('time', '>f8'), ('glon', '>f8'), ('glat', '>f8'),
                           ('zg', '>f4'), ('rh25', '>f4'), ('rh50', '>f4'), ('rh75', '>f4'), ('rh100', '>f4')])
        elif version == '1.03':
            dt = np.dtype(
                [('lfid', '>u4'), ('shotnumber', '>u4'), ('azimuth', '>f4'), ('incidentangle', '>f4'), ('range', '>f4'),
                 ('time', '>f8'), ('glon', '>f8'), ('glat', '>f8'), ('zg', '>f4'), ('rh25', '>f4'), ('rh50', '>f4'),
                 ('rh75', '>f4'), ('rh100', '>f4')])
        elif version == '1.04':
            dt = np.dtype(
                [('lfid', '<u4'), ('shotnumber', '<u4'), ('azimuth', '<f4'), ('incidentangle', '<f4'), ('range', '<f4'),
                 ('time', '<f8'), ('glon', '<f8'), ('glat', '<f8'), ('zg', '<f4'), ('rh25', '<f4'), ('rh50', '<f4'),
                 ('rh75', '<f4'), ('rh100', '<f4')])


    elif filetype == 'lgw':
        if version == '1.00':
            dt = np.dtype(
                [('lon0', '>f8'), ('lat0', '>f8'), ('z0', '>f4'), ('lon431', '>f8'), ('lat431', '>f8'), ('z431', '>f4'),
                 ('sigmean', '>f4'), ('wave', '>u1', 432)])
        elif version == '1.01':
            dt = np.dtype([('lfid', '>u4'), ('shotnumber', '>u4'), ('lon0', '>f8'), ('lat0', '>f8'), ('z0', '>f4'),
                           ('lon431', '>f8'), ('lat431', '>f8'), ('z431', '>f4'), ('sigmean', '>f4'),
                           ('wave', '>u1', 432)])
        elif version == '1.02':
            dt = np.dtype([('lfid', '>u4'), ('shotnumber', '>u4'), ('time', '>f8'), ('lon0', '>f8'), ('lat0', '>f8'),
                           ('z0', '>f4'), ('lon431', '>f8'), ('lat431', '>f8'), ('z431', '>f4'), ('sigmean', '>f4'),
                           ('wave', '>u1', 432)])
        elif version == '1.03':
            dt = np.dtype(
                [('lfid', '>u4'), ('shotnumber', '>u4'), ('azimuth', '>f4'), ('incidentangle', '>f4'), ('range', '>f4'),
                 ('time', '>f8'), ('lon0', '>f8'), ('lat0', '>f8'), ('z0', '>f4'), ('lon431', '>f8'), ('lat431', '>f8'),
                 ('z431', '>f4'), ('sigmean', '>f4'), ('txwave', '>u1', 80), ('rxwave', '>u1', 432)])
        elif version == '1.04':
            dt = np.dtype(
                [('lfid', '<u4'), ('shotnumber', '<u4'), ('azimuth', '<f4'), ('incidentangle', '<f4'), ('range', '<f4'),
                 ('time', '<f8'), ('lon0', '<f8'), ('lat0', '<f8'), ('z0', '<f4'), ('lon431', '<f8'), ('lat431', '<f8'),
                 ('z431', '<f4'), ('sigmean', '<f4'), ('txwave', '<u2', 120), ('rxwave', '<u2', 528)])

    return dt


def read_legacy_lvis(input_file, version):
    if input_file.find('lce') != -1:
        filetype = 'lce'
    elif input_file.find('lge') != -1:
        filetype = 'lge'
    elif input_file.find('lgw') != -1:
        filetype = 'lgw'
    else:
        print('Unrecognized file type.')
        return

    if not version in ['1.00', '1.01', '1.02', '1.03', '1.04']:
        print('Unrecognized data version.')
        return

    try:
        dt = get_datatype(filetype, version)
        x = np.fromfile(input_file, dt)

        # Sanity check lat/lon - catch version number or byte order problems
        if filetype == 'lce':
            l = x['tlat'][np.where(x['tlat'] != 0)]
            if max(np.log10(np.abs(l))) > 2 or min(np.log10(np.abs(l))) < -12:
                # Absurd value in latitude. Try flipping the byte order. If it's still bad,
                # likely that file version number was wrong.
                if max(np.log10(np.abs(l.newbyteorder()))) > 2 or min(np.log10(np.abs(l.newbyteorder()))) < -12:
                    print('Warning: Bad latitude values. Check LVIS data version!')
                else:
                    z = x.newbyteorder()
                    return z
            else:
                pass
        elif filetype == 'lge':
            l = x['glat'][np.where(x['glat'] != 0)]
            if max(np.log10(abs(l))) > 2 or min(np.log10(abs(l))) < -12:
                if max(np.log10(np.abs(l.newbyteorder()))) > 2 or min(np.log10(np.abs(l.newbyteorder()))) < -12:
                    print('Warning: Bad latitude values. Check LVIS data version!')
                else:
                    z = x.newbyteorder()
                    return z
            else:
                pass
        elif filetype == 'lgw':
            l = x['lat0'][np.where(x['lat0'] != 0)]
            if max(np.log10(abs(l))) > 2 or min(np.log10(abs(l))) < -12:
                if max(np.log10(np.abs(l.newbyteorder()))) > 2 or min(np.log10(np.abs(l.newbyteorder()))) < -12:
                    print('Warning: Bad latitude values. Check LVIS data version!')
                else:
                    z = x.newbyteorder()
                    return z
            else:
                pass

        return x
    except IOError:
        print('Legacy LVIS file does not exist or is not readable.')
        return
#Biomass index
test = read_legacy_lvis(test, '1.02')
test =  np.array(test)
test1 = read_legacy_lvis(test1, '1.02')
test2 = read_legacy_lvis(test2, '1.02')

#Reading get wave index from shotnumber and test if exists
all_shotnums = np.array(test['shotnumber'])
wave_idx = ''
Zmax = ''
wfrange = 432
wfsize = 431


#With it read in next part is to read on of the shots - THis is where the loop needs to be.
myshotnum = int(3260613)

#The bumps the selected shot again the shot list-RA MAy
wave_idx = np.where(all_shotnums == myshotnum)

#extract the single waveform and elevation attributes Z0 and Z1023;
waveform = np.array(test2['wave'][wave_idx])

#This extracting the Z0
Z0 =  np.array(test2['z0'][wave_idx])

#This extracting the Z1023
Z1023 = np.array(test2['z431'][wave_idx])

ZG =np.array(test1['zg'][wave_idx])

ZT = np.array(test['zt'][wave_idx])

RH10 = np.array(test1['zg'][wave_idx])

RH100 = np.array(test1['rh100'][wave_idx])

#Check if this does anything
x = Z0 - Z1023

#find the elevation difference from Z0 to Z1023 and divide into 1023 equal intervals
zstretch = np.add(Z1023,np.multiply(range(wfrange,0,-1),((Z0-Z1023)/int(wfsize))))


# set the z range limits for plotting the waveform to crop the noise
zmin = RH10 - (ZT - RH10) / 6  # sets zmin at 15% below the waveform range defined in RH10-ZT
zmax = ZT + (ZT - RH10) / 18  # sets zmax at 5% above the waveform range defined in RH10-ZT


#This subtract with the ground height normalizing everything
zstretch = zstretch - ZG

# crop the waveform and elevation arrays to the z range limits
x = zstretch >= zmin  # this returns boolean True/False based on the comparison condition statement
y = zstretch <= zmax
z = (x == y)  # this creates a combined boolean result from the previous two condition statements
waveform_crop = []
zstretch_crop = []
for i in range(0, len(waveform)):
    if z[i] == True:
        waveform_crop.append(waveform[i])
        zstretch_crop.append(zstretch[i])


SIGMEAN = np.array(test2['sigmean'][wave_idx])

cal_wave = np.subtract(waveform , SIGMEAN)
#Look up what reshape does - 020921
cal_wave = cal_wave.reshape(cal_wave.size,1)

X = np.arange(432)
X = X.reshape(X.size,1)


#elevation
wz = X*((Z1023 - Z0)/432) +Z0
#Height
wz = np.subtract(wz, ZG)


filter_arr = np.logical_and(-10 < wz, wz < RH100)

ind = wz[filter_arr]

wz = wz[filter_arr]

#Should be a table of 1000
bio = 0
sub_wz = wz # -
sub_cal_wave = cal_wave[filter_arr] # -
sum_count = np.sum(sub_cal_wave) #
iii=0

#that reads everything
for iii , elements in  enumerate(sub_cal_wave):
    #print(bio)
    #print(iii)
    #print(iii+1)
    #print(sub_cal_wave[iii]/sum_count)
    #dp[iii] = (sub_cal_wave[iii]/sum_count)
    if (sub_wz[iii] > - 10) and (sub_wz[iii] < 0) :
        bio = bio + sub_wz[iii] * sub_wz[iii] * (sub_cal_wave[iii]/sum_count) * (-1.0)
    else:
        bio = bio + sub_wz[iii] * sub_wz[iii] * (sub_cal_wave[iii]/sum_count)
        #print('This is the bio')
        #print(bio)
        #print(myshotnum)

print(bio)
print(myshotnum)


55.361615485355905
3260613
